In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [17]:
load_dotenv(override=True) #Why ?

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"Bạn đã đăng nhập được khóa với mã {openai_api_key[:8]}")
else:
    print("Không tìm thấy mã khóa")

MODEL = "gpt-4.1-mini"
openai = OpenAI()

Bạn đã đăng nhập được khóa với mã sk-proj-


In [ ]:
system_message = """
Bạn là một người bán hàng đang hỗ trợ cửa hàng bán quần áo . 
Nhiệm vụ của bạn là thông báo cho khách hàng những sản phầm có bên trong cửa hàng, và hãy trả lời tôi không biết khi bạn không biết thông tin gì về món hàng đó trong dữ liệu được lấy từ data của cửa hàng hoặc khi khách hàng hỏi về một thông tin khác ngoài dữ liệu trong cửa hàng 
Không trả lời những câu hỏi ngoài khía cạnh buôn bán của cửa hàng. Tất cả các giá khi được khách hàng nhắc đến đều quy về tiền VNĐ hết " 
"""

In [19]:
# Kiểm tra cơ bản hệ thống
def basic_chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )
    return response.choices[0].message.content

In [20]:
gr.ChatInterface(fn=basic_chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [21]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_products",
    "description": "Lấy giá và trả về tên quần áo .",
    "parameters": {
        "type": "object",
        "properties": {
            "name_of_product": {
                "type": "string",
                "description": "Cái quần áo cái mà khách hàng muốn",
            },
        },
        "required": ["name_of_product"],
        "additionalProperties": False
    }
}



In [22]:
#Dummy data
products = {
    "áo thun": "50.000",
    "quần tây": "70.000",
    "áo khoác": "100.000",
    "quần thể thao": "80.000"
}

def get_products (name_of_product):
    print(f"Bạn đang lấy sản phẩm {name_of_product}")
    price = products.get(name_of_product.lower(), f"Tôi không có sản phẩm này")
    return f"Giá của sản phẩm {name_of_product} là {price}"

print(get_products(f"Quần tây"))

Bạn đang lấy sản phẩm Quần tây
Giá của sản phẩm Quần tây là 70.000


In [23]:
tools = [{"type": "function", "function": price_function}]

tools

[{'type': 'function',
  'function': {'name': 'get_products',
   'description': 'Lấy giá và trả về tên quần áo .',
   'parameters': {'type': 'object',
    'properties': {'name_of_product': {'type': 'string',
      'description': 'Cái quần áo cái mà khách hàng muốn'}},
    'required': ['name_of_product'],
    'additionalProperties': False}}}]

In [24]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_products":
        arguments = json.loads(tool_call.function.arguments)
        product = arguments.get('name_of_product')
        price_details = get_products(product)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [25]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [26]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [34]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    tool_call = message.tool_calls[0]
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_products":
            arguments = json.loads(tool_call.function.arguments)
            product = arguments.get('name_of_product')
            price_details = get_products(product)
            responses.append ({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [35]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [36]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [44]:
import sqlite3

In [68]:
DB = "prices.db"
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS prices')
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (cloth TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [73]:
def get_products(cloth):
    print(f"DATABASE TOOL CALLED: Getting price for {cloth}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE cloth = ?', (cloth.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {cloth} is ${result[0]}" if result else "Không có thông tin về giá quần áo này"

In [69]:

def set_price(cloth, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (cloth, price) VALUES (?, ?) ON CONFLICT(cloth) DO UPDATE SET price = ?', (cloth.lower(), price, price))
        conn.commit()

In [70]:
clothing_prices = {
    # Áo Nam
    "Áo thun nam Basic": 150000,
    "Áo sơ mi công sở nam": 350000,
    "Áo Polo nam": 220000,
    "Áo ba lỗ nam": 70000,
    "Áo khoác gió nam": 450000,
    "Vest nam cao cấp": 2500000,
    
    # Quần Nam
    "Quần Jean nam rách": 420000,
    "Quần Tây âu": 380000,
    "Quần Short Kaki": 180000,
    "Quần Jogger thun": 250000,

    # Áo Nữ
    "Áo Croptop nữ": 120000,
    "Áo sơ mi lụa nữ": 320000,
    "Áo Blazer nữ": 550000,
    "Áo len cổ lọ": 280000,
    "Áo hai dây": 80000,

    # Quần/Váy Nữ
    "Chân váy chữ A": 190000,
    "Đầm Maxi đi biển": 450000,
    "Quần Legging": 110000,
    "Quần Culottes ống rộng": 260000,
    "Váy liền thân hoa nhí": 340000,
    
    # Unisex / Phụ kiện
    "Áo Hoodie Unisex": 350000,
    "Áo Sweater nỉ": 290000
}

In [71]:
for cloth, price in clothing_prices.items():
    set_price(cloth, price)

In [76]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
